## Enviroment

In [2]:
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy_utils import database_exists, create_database
from dotenv import load_dotenv
import os
import logging # Mantendremos un logging mínimo para información útil

# Configuración básica de logging (opcional, puedes quitarla si prefieres solo prints)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S')

In [3]:
# Cargar variables de entorno desde el archivo .env
# Ajusta la ruta si tu archivo .env está en otro lugar
dotenv_path = os.path.join(os.getcwd(), '..', 'env', '.env')
if not os.path.exists(dotenv_path):
    # Intenta buscar en el directorio actual si no está en ../env/
    dotenv_path = os.path.join(os.getcwd(), '.env')
    if not os.path.exists(dotenv_path):
         logging.warning(f"Archivo .env no encontrado en {os.path.join(os.getcwd(), '..', 'env', '.env')} ni en {os.getcwd()}. Asegúrate de que exista y contenga las credenciales.")
         # Podrías detener la ejecución aquí si es crítico
         # raise FileNotFoundError("Archivo .env no encontrado.")


if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path)
    logging.info(f"Variables de entorno cargadas desde: {dotenv_path}")
else:
    logging.warning("No se encontró el archivo .env en las rutas esperadas. Intentando usar variables de entorno del sistema si existen.")
    # load_dotenv() # Descomentar si quieres intentar cargar desde variables del sistema


# Obtener credenciales desde las variables de entorno
DB_USER = os.getenv("USER")
DB_PASSWORD = os.getenv("PASSWORD")
DB_HOST = os.getenv("HOST")
DB_PORT = os.getenv("PORT")
DB_DATABASE = os.getenv("DATABASE")

# Validar que las variables se cargaron (simple verificación)
if not all([DB_USER, DB_PASSWORD, DB_HOST, DB_PORT, DB_DATABASE]):
    raise ValueError("Una o más variables de entorno (USER, PASSWORD, HOST, PORT, DATABASE) no están definidas. Verifica tu archivo .env o las variables del sistema.")

# Definir rutas a los archivos (ajusta si es necesario)
# Se asume que el notebook está en un directorio y 'data' está un nivel arriba
CSV_PATH = os.path.join(os.getcwd(), "..", "data", "Airbnb_Open_Data.csv")
TABLE_NAME = "airbnb_data_raw" # Usamos un nombre diferente para la tabla cruda

logging.info(f"Ruta CSV: {CSV_PATH}")
logging.info(f"Nombre de tabla: {TABLE_NAME}")
logging.info(f"Base de datos destino: {DB_DATABASE}")

2025-04-07 02:43:09 - INFO - Variables de entorno cargadas desde: /home/y4xul/ETL/PROYECTO/Proyecto_ETL/Notebooks/../env/.env
2025-04-07 02:43:09 - INFO - Ruta CSV: /home/y4xul/ETL/PROYECTO/Proyecto_ETL/Notebooks/../data/Airbnb_Open_Data.csv
2025-04-07 02:43:09 - INFO - Nombre de tabla: airbnb_data_raw
2025-04-07 02:43:09 - INFO - Base de datos destino: airbnb


In [5]:
# Construir URLs de conexión
# URL para conectar al servidor PostgreSQL (usando 'postgres' db por defecto para verificar/crear)
server_url = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/postgres"
# URL para conectar a la base de datos específica de destino
db_url = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_DATABASE}"

# Verificar si la base de datos existe
try:
    if not database_exists(db_url):
        logging.info(f"La base de datos '{DB_DATABASE}' no existe. Creándola...")
        # Crear la base de datos (necesita conexión al servidor, no a la DB específica)
        create_database(db_url)
        logging.info(f"Base de datos '{DB_DATABASE}' creada.")
    else:
        logging.info(f"La base de datos '{DB_DATABASE}' ya existe.")

    # Crear el engine para conectar a la base de datos específica
    engine = create_engine(db_url)
    logging.info(f"Engine creado para la base de datos '{DB_DATABASE}'.")

    # Probar conexión simple (opcional)
    with engine.connect() as connection:
        connection.execute(text("SELECT 1"))
    logging.info("Conexión a la base de datos exitosa.")

except Exception as e:
    logging.error(f"Error durante la configuración de la base de datos o creación del engine: {e}")
    # En un notebook, podrías querer detener aquí o manejar el error
    raise # Relanzar el error para detener la ejecución del notebook si falla la conexión/creación

2025-04-07 02:48:15 - INFO - La base de datos 'airbnb' no existe. Creándola...
2025-04-07 02:48:15 - ERROR - Error durante la configuración de la base de datos o creación del engine: (psycopg2.OperationalError) connection to server at "localhost" (127.0.0.1), port 5432 failed: FATAL:  password authentication failed for user "y4xul"
connection to server at "localhost" (127.0.0.1), port 5432 failed: FATAL:  password authentication failed for user "y4xul"

(Background on this error at: https://sqlalche.me/e/14/e3q8)


OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (127.0.0.1), port 5432 failed: FATAL:  password authentication failed for user "y4xul"
connection to server at "localhost" (127.0.0.1), port 5432 failed: FATAL:  password authentication failed for user "y4xul"

(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
# Cargar el archivo CSV en un DataFrame de pandas
try:
    df = pd.read_csv(CSV_PATH, low_memory=False, encoding='ISO-8859-1')
    logging.info(f"Archivo CSV '{os.path.basename(CSV_PATH)}' cargado. Filas: {len(df)}")

    # Mostrar las primeras filas para verificar
    print("\nPrimeras 5 filas del DataFrame:")
    display(df.head()) # 'display' es mejor que 'print' para DataFrames en notebooks

except FileNotFoundError:
    logging.error(f"Error: Archivo CSV no encontrado en la ruta: {CSV_PATH}")
    # Detener si el archivo no se encuentra
    raise
except Exception as e:
    logging.error(f"Error al leer el archivo CSV: {e}")
    raise

In [ ]:
# Cargar el DataFrame a la tabla especificada en la base de datos
# if_exists='replace' borrará la tabla si existe y la creará de nuevo
try:
    logging.info(f"Cargando datos en la tabla '{TABLE_NAME}' (esto puede tardar)...")
    df.to_sql(TABLE_NAME, engine, if_exists='replace', index=False, chunksize=10000) # chunksize ayuda con memoria
    logging.info(f"Datos cargados exitosamente en la tabla '{TABLE_NAME}'.")

except Exception as e:
    logging.error(f"Error al cargar datos en la base de datos: {e}")
    raise

In [ ]:
# Verificar que los datos se cargaron contando las filas en la tabla
try:
    with engine.connect() as connection:
        result = connection.execute(text(f"SELECT COUNT(*) FROM {TABLE_NAME};"))
        db_rows_count = result.scalar() # Obtener el primer valor de la primera fila

    logging.info(f"Verificación: Número de filas en CSV = {len(df)}")
    logging.info(f"Verificación: Número de filas en tabla '{TABLE_NAME}' = {db_rows_count}")

    if len(df) == db_rows_count:
        print("\n¡Éxito! El número de filas coincide.")
    else:
        print("\n¡Advertencia! El número de filas en el CSV y en la tabla no coincide.")

    # Opcional: Mostrar las primeras 5 filas desde la BD
    # print(f"\nPrimeras 5 filas leídas desde la tabla '{TABLE_NAME}':")
    # query_head = f"SELECT * FROM {TABLE_NAME} LIMIT 5;"
    # df_from_db = pd.read_sql(query_head, engine)
    # display(df_from_db)

except Exception as e:
    logging.error(f"Error durante la verificación de datos en la base de datos: {e}")

# Nota: En un notebook no es estrictamente necesario hacer engine.dispose()
# ya que el kernel maneja los recursos, pero si ejecutas muchas veces
# o tienes operaciones muy largas, podría ser buena práctica al final.
# Ejemplo:
# if 'engine' in locals() and engine:
#     engine.dispose()
#     logging.info("Engine dispuesto.")

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text
import json
import logging

In [ ]:
df = pd.read_csv("../data/Airbnb_Open_Data.csv", low_memory=False, encoding='ISO-8859-1')
print("Datos del CSV cargados correctamente.")
print(df.head())

Datos del CSV cargados correctamente.
        id                                              NAME      host id  \
0  1001254                Clean & quiet apt home by the park  80014485718   
1  1002102                             Skylit Midtown Castle  52335172823   
2  1002403               THE VILLAGE OF HARLEM....NEW YORK !  78829239556   
3  1002755                                               NaN  85098326012   
4  1003689  Entire Apt: Spacious Studio/Loft by central park  92037596077   

  host_identity_verified host name neighbourhood group neighbourhood  \
0            unconfirmed  Madaline            Brooklyn    Kensington   
1               verified     Jenna           Manhattan       Midtown   
2                    NaN     Elise           Manhattan        Harlem   
3            unconfirmed     Garry            Brooklyn  Clinton Hill   
4               verified    Lyndon           Manhattan   East Harlem   

        lat      long        country  ... service fee minimum nigh

Conexión a la base de datos mediante las credenciales definidas

In [ ]:
def create_db_engine():
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    credentials = "../credentials.json"

    """
    Lee las credenciales desde un archivo JSON, crea y retorna un engine para conectarse a la base de datos.
    Utiliza logging para notificar el resultado de la conexión.
    """
    try:
        with open(credentials, 'r') as file:
            config = json.load(file)
        user = config["user"]
        password = config["password"]
        host = config["host"]
        port = config["port"]
        database = config["database"]
        
        # Construcción de la cadena de conexión
        connection_string = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}"
        engine = create_engine(connection_string)
        
        # Realizamos una consulta simple para probar la conexión
        with engine.connect() as conn:
            conn.execute(text("SELECT 1"))
        logging.info("Conexión a la base de datos exitosa.")
        
        return engine
    except Exception as e:
        logging.error("Error al conectar a la base de datos: %s", e)
        return None

# Ejemplo de uso:
if __name__ == "__main__":
    engine = create_db_engine()
    if engine:
        logging.info("El engine se creó correctamente y está listo para usar.")


2025-03-21 09:38:16,347 - INFO - Conexión a la base de datos exitosa.
2025-03-21 09:38:16,347 - INFO - El engine se creó correctamente y está listo para usar.


In [ ]:
credentials = "../credentials.json"

with open(credentials) as f:
    creds = json.load(f)
    
DB_USER = creds["user"]
DB_PASSWORD = creds["password"]
DB_HOST = creds["host"]
DB_PORT = creds["port"]
DB_NAME = creds["database"]

creator_connection_string = f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/postgres'
creator_engine = create_engine(creator_connection_string)

with creator_engine.connect() as connection:
    connection.execution_options(isolation_level="AUTOCOMMIT")
    result = connection.execute(
        text(f"SELECT 1 FROM pg_database WHERE datname = '{DB_NAME}'")
    )
    
    if not result.scalar():
        connection.execute(text(f"CREATE DATABASE {DB_NAME}"))
        print(f"Base de datos '{DB_NAME}' creada exitosamente!")
    else:
        print(f"La base de datos '{DB_NAME}' ya existe.")

La base de datos 'airbnb' ya existe.


In [ ]:
connection_string = f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
engine = create_engine(connection_string)

table_name = "airbnb_data"
df.to_sql(table_name, engine, if_exists='replace', index=False)

print(f"Tabla '{table_name}' creada y datos cargados en la base de datos '{DB_NAME}'.")

Tabla 'airbnb_data' creada y datos cargados en la base de datos 'airbnb'.


In [ ]:
with engine.connect() as connection:
    query = f"SELECT * FROM {table_name} LIMIT 5;"
    df_5filas = pd.read_sql(query, connection)
    print(f"Primeros 5 registros de la tabla '{table_name}':")
    print(df_5filas)

Primeros 5 registros de la tabla 'airbnb_data':
        id                                              NAME      host id  \
0  1001254                Clean & quiet apt home by the park  80014485718   
1  1002102                             Skylit Midtown Castle  52335172823   
2  1002403               THE VILLAGE OF HARLEM....NEW YORK !  78829239556   
3  1002755                                              None  85098326012   
4  1003689  Entire Apt: Spacious Studio/Loft by central park  92037596077   

  host_identity_verified host name neighbourhood group neighbourhood  \
0            unconfirmed  Madaline            Brooklyn    Kensington   
1               verified     Jenna           Manhattan       Midtown   
2                   None     Elise           Manhattan        Harlem   
3            unconfirmed     Garry            Brooklyn  Clinton Hill   
4               verified    Lyndon           Manhattan   East Harlem   

        lat      long        country  ... service fee  m

In [ ]:
total_csv_rows = len(df)
print(f"El archivo CSV tiene un total de {total_csv_rows} filas.")

with engine.connect() as connection:
    result = connection.execute(text(f"SELECT COUNT(*) FROM {table_name};"))
    total_rows = result.scalar()
    print(f"La tabla '{table_name}' tiene un total de {total_rows} filas.")

El archivo CSV tiene un total de 102599 filas.
La tabla 'airbnb_data' tiene un total de 102599 filas.
